In [ ]:
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

In [ ]:
num_speaker = 2

language = 'English'
model_size = 'large'

model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

In [ ]:
import locale
def getpreferredencoding(do_setlocal = True):
  return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np


In [ ]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

In [ ]:
model = whisper.load_model(model_size)

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [ ]:
audio = Audio()

In [ ]:
def segment_embedding(segment):
  start = segment['start']
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  waveform = waveform.mean(dim=0, keepdim=True)
  return embedding_model(waveform.unsqueeze(0))

In [ ]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)


embeddings = np.nan_to_num(embeddings)

In [ ]:
clustering = AgglomerativeClustering(num_speaker).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w",encoding="UTF-8")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [ ]:
print(open('transcript.txt','r',encoding="UTF-8").read())